In [1]:
import numpy as np
from pyprojroot.here import here
import pandas as pd
import janitor
data = janitor.clean_names(pd.read_excel(here("data/por-municipios.xlsx")))
data 

C:\Users\Plancha\Desktop\pfacd\.pixi\envs\default\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()


,year,month,date,district,municipality,parish,voltage_level,active_energy_kwh_,districtcode,districtmunicipalitycode,districtmunicipalityparishcode
0,2023,5,2023-05,SANTAREM,Rio Maior,UF AZAMBUJEIRA E MALAQUEIJO,"Muito Alta, Alta e Média Tensões",42889.398,14,1414,141415
1,2023,5,2023-05,SETUBAL,Sesimbra,SESIMBRA (SANTIAGO),"Muito Alta, Alta e Média Tensões",501913.130,15,1511,151102
2,2023,5,2023-05,VILA REAL,Murça,JOU,Baixa Tensão,70955.155,17,1707,170704
3,2023,5,2023-05,VILA REAL,Santa Marta de Penaguião,SEVER,"Muito Alta, Alta e Média Tensões",52292.716,17,1711,171110
4,2023,5,2023-05,VISEU,Armamar,SAO MARTINHO DAS CHAS,Baixa Tensão,95161.235,18,1801,180115
...,...,...,...,...,...,...,...,...,...,...,...
165019,2022,7,2022-07,BRAGANCA,Vinhais,VINHAIS,"Muito Alta, Alta e Média Tensões",63732.258,4,412,041235
165020,2022,7,2022-07,COIMBRA,Figueira da Foz,MARINHA DAS ONDAS,Baixa Tensão,434881.302,6,605,060508
165021,2022,7,2022-07,FARO,Alcoutim,MARTIM LONGO,Baixa Tensão,162904.786,8,802,080203
165022,2022,7,2022-07,LEIRIA,Pedrógão Grande,GRACA,Baixa Tensão,107855.195,10,1013,101301


In [45]:
from datar.dplyr import distinct, arrange, group_by, summarise, n, rename, select, c_across, nth, filter_
from datar import f

parish_code_count = (data >> 
 group_by(f.parish, f.districtmunicipalityparishcode, f.date) >> 
 summarise(
     n = n(),
     first_energy = nth(f.active_energy_kwh_, 0),
     last_energy = nth(f.active_energy_kwh_, 1)
 ) >>
 rename(parish_code = f.districtmunicipalityparishcode)
)
parish_code_count

C:\Users\Plancha\Desktop\pfacd\.pixi\envs\default\Lib\site-packages\datar_pandas\tibble.py:406: FutureWarning: DataFrameGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  "group_vars", self._datar["grouped"].grouper.names
C:\Users\Plancha\Desktop\pfacd\.pixi\envs\default\Lib\site-packages\datar_pandas\api\dplyr\group_data.py:56: FutureWarning: DataFrameGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  grouper = _data._datar["grouped"].grouper
C:\Users\Plancha\Desktop\pfacd\.pixi\envs\default\Lib\site-packages\datar_pandas\api\dplyr\summarise.py:105: FutureWarning: DataFrameGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  grouped.grouper.names,
C:\Users\Plancha\Desktop\pfacd\.pixi\envs\default\Lib\site-packages\datar_pandas\tibble.py:351: FutureWarning: DataFrameGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  grouped.grouper,
C:\Users\Plancha\Desktop\pfac

,parish,parish_code,date,n,first_energy,last_energy
,<object>,<object>,<object>,<int64>,<float64>,<float64>
0,UF AZAMBUJEIRA E MALAQUEIJO,141415,2023-05,2,42889.398,175872.824
1,SESIMBRA (SANTIAGO),151102,2023-05,2,501913.130,1179419.238
2,JOU,170704,2023-05,1,70955.155,NaN
3,SEVER,171110,2023-05,2,52292.716,55327.776
...,...,...,...,...,...,...
4,SAO MARTINHO DAS CHAS,180115,2023-05,2,95161.235,11132.964
5,SAZES DA BEIRA,091219,2022-05,1,31541.195,NaN
6,NUMAO,091411,2022-05,1,27459.619,NaN
7,URRO,010419,2022-06,1,106768.910,NaN


In [55]:
(parish_code_count >>
 filter_(f.n > 1) >>
 select(f.parish, f.parish_code, f.date, f.first_energy, f.last_energy) >>
 arrange(f.parish_code)
)

C:\Users\Plancha\Desktop\pfacd\.pixi\envs\default\Lib\site-packages\datar_pandas\broadcast.py:684: FutureWarning: SeriesGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  return get_obj(value).index, value.grouper
C:\Users\Plancha\Desktop\pfacd\.pixi\envs\default\Lib\site-packages\datar_pandas\broadcast.py:684: FutureWarning: SeriesGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  return get_obj(value).index, value.grouper
C:\Users\Plancha\Desktop\pfacd\.pixi\envs\default\Lib\site-packages\datar_pandas\api\dplyr\filter_.py:37: FutureWarning: DataFrameGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  grouper = _data._datar["grouped"].grouper
C:\Users\Plancha\Desktop\pfacd\.pixi\envs\default\Lib\site-packages\datar_pandas\broadcast.py:633: FutureWarning: SeriesGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  if np.array_equal(grouper.group_info[0], value.gro

,parish,parish_code,date,first_energy,last_energy
,<object>,<object>,<object>,<float64>,<float64>
0,AGUADA DE CIMA,010103,2022-06,610380.950,2014952.890
1,AGUADA DE CIMA,010103,2022-02,2192026.527,711071.207
2,AGUADA DE CIMA,010103,2021-09,635471.220,1997839.065
3,AGUADA DE CIMA,010103,2021-08,1582618.409,634542.016
...,...,...,...,...,...
4,AGUADA DE CIMA,010103,2023-09,620391.485,1436024.889
5,BRAGA,NaN,2021-07,4775490.824,8484.711
6,BRAGA,NaN,2022-10,3838833.279,9451.662
7,BRAGA,NaN,2021-04,8608.839,4547294.017


In [59]:
from datar.base import grepl

pcc = pd.DataFrame(parish_code_count)
pcc.loc[~grepl(r"\d\d\d\d\d\d", pcc.parish_code), :]

,parish,parish_code,date,n,first_energy,last_energy
,<object>,<object>,<object>,<int64>,<float64>,<float64>
54,UF SILVEIROS E RIO COVO,0302FE,2020-12,2,220343.825,764479.434
306,BRAGANCA,NaN,2022-09,1,1592032.703,NaN
307,LEIRIA,NaN,2021-01,1,424767.852,NaN
601,UF VILA COVA E FEITOS,0302FH,2020-11,2,30892.035,320573.732
...,...,...,...,...,...,...
853,VIANA DO CASTELO,NaN,2023-11,1,793447.660,NaN
106883,FARO,NaN,2020-11,1,116700.611,NaN
106971,UF QUINTIAES E AGUIAR,0302FC,2021-07,1,183766.045,NaN
107007,UF QUINTIAES E AGUIAR,0302FC,2022-09,1,133894.320,NaN


2 tipos de erros de parish:
- repetidos (mesma data e id com energias diferentes)
- Concelho na freguesia (se calhar e a media?)

Os ids estao em base maior que 10 ptt e preciso transferir isso tmb se for necessario

In [50]:
import re
pattern = re.compile(r"\d\d\d\d\d\d")
pattern.match("010101")

<re.Match object; span=(0, 6), match='010101'>